# Reconnaissance de signatures : apprentisage et validation de véracité

### Import des librairies

In [1]:
import pygame
from pygame.locals import *;
from math import sin, cos, atan2, sqrt, pi
from xml.dom import minidom
import pandas as pd
import pprint
import os.path

- pygame : Librairie de jeux vidéo, président ici pour la gesture
- math import sin, cos, atan2, sqrt, pi : Fonctions mathématiques
- xml.dom import minidom : pour naviguer dans le DOM, ici il va servir pour parser
- pandas as pd : Sert ici pour écrire, lire et fusionner des dataFrame ansi que des csv
- numpy as np : Utile ici uniquement pour faire des tableaux
- pprint : pour printer le 1er tableau qui est le dataset de signature sous forme de nuage de point
- import os.path : Utile ici pour vérifier que les fichiers CSV finaux sont bien présent ou non

### Variables super globales à renseigner (1ère lettre en majuscule)

In [2]:
Nom = "test"

Prenom = "William"

#Taille des signatures, ex : 250 si fenetre de 500 généralement.
Size = 250

#Précision de la véracité en pourcent,
#sachant que plus on s'approche de 0% plus la signature une copie conforme.
#Si vous choisissez 30%/200% par exemple, 
#cela signifit que la moyenne de tous les pourcentages de véracité du dataset est inférieur à 20%.
Precision = 70

### Création d'un jeu de donnée saisie directement de l'utilisateur

In [3]:
pygame.init();
screen = pygame.display.set_mode((500,500));
pygame.display.flip();
screen.fill((0,0,0))
running = True
recording = False
points = []
signatures = []
while running:
    events = pygame.event.get();
    for e in events:
        if e.type==MOUSEBUTTONDOWN:
            screen.fill((0,0,0))
            points = []
            recording = True
        if e.type==MOUSEBUTTONUP:
            recording = False
            if len(points) > 10:
                signatures.append(points)
        if e.type==MOUSEMOTION:
            if recording:
                mousepos= pygame.mouse.get_pos()
                points.append(list(mousepos))
                pygame.draw.circle(screen,(255,255,255), mousepos, 3);
        if e.type==QUIT:
            running = 0
    pygame.time.wait(2);
    pygame.display.flip();
pp = pprint.PrettyPrinter(indent=2)
print "Jeu de données brut :"
pp.pprint(signatures)

Jeu de données brut :
[ [ [227, 132],
    [231, 150],
    [233, 170],
    [235, 185],
    [237, 200],
    [238, 208],
    [240, 214],
    [241, 216],
    [242, 217],
    [243, 217],
    [250, 217],
    [274, 199],
    [306, 172],
    [323, 154],
    [335, 138],
    [342, 127],
    [345, 121],
    [348, 118],
    [349, 117],
    [349, 116],
    [349, 118],
    [349, 126],
    [349, 138],
    [348, 156],
    [346, 171],
    [344, 181],
    [343, 187],
    [341, 191],
    [341, 192],
    [340, 192],
    [340, 187],
    [341, 179],
    [344, 173],
    [347, 165],
    [352, 158],
    [355, 154],
    [357, 152],
    [360, 148],
    [365, 142],
    [377, 121]],
  [ [285, 115],
    [285, 122],
    [287, 132],
    [287, 142],
    [287, 148],
    [287, 152],
    [287, 153],
    [287, 154],
    [288, 154],
    [289, 149],
    [291, 139],
    [294, 130],
    [295, 125],
    [297, 121],
    [298, 118],
    [298, 117],
    [299, 117],
    [299, 116],
    [299, 121],
    [299, 128],
    [299, 138],
 

### Jeu de données de points représentant une signature

In [4]:
class Template:
    def __init__(self, name, points):
        self.points = points
        self.name = name

    def prepare(self):
        self.points = resample(self.points, 64)
        self.points = rotate_to_zero(self.points)
        self.points = scale_to_square(self.points, 250)
        self.points = translate_to_origin(self.points)

templates = []
i = 0
#Récupération du jeu de donnée CSV BDD
if os.path.isfile('bddSignature.csv') :
    #parcours du CSV
    dfbdd = pd.read_csv('bddSignature.csv', sep = ';')
    for indice, valeur in enumerate(dfbdd['Signature du dataset']):
        if dfbdd['Nom'][indice] == Nom and dfbdd['Prénom'][indice] == Prenom :
            if valeur <> '>' :
                valeur = valeur.replace('[[', '').replace('[', '').replace(',' , '').replace(']', '').replace(']]', '')
                valeur = valeur.split(' ')
                one_list = []
                two_list = []
                for z in range(len(valeur)):                
                    if z % 2 != 0:
                        one_list.append(valeur[z])
                    else : two_list.append(valeur[z])
                tuple_list = []
                for w in range(len(valeur)/2):
                    tuple_list.append([float(one_list[w]),float(two_list[w])])
                templates.append(Template("numéro "+`i`+" de "+Nom+" "+Prenom, tuple_list))
        i = i+1
for s in signatures:
    #utile pour le débug : résumer du template
    i = i+1
    templates.append(Template("numéro "+`i`+" de "+Nom+" "+Prenom, s))

### Fonctions servant à la reconnaisance

In [5]:
def average(xs): return sum(xs) / len(xs)

def resample(points, n):#{{{
    I = pathlength(points) / float(n-1)
    D = 0
    newPoints = [points[0]]
    i = 1
    while i<len(points):
        p_i = points[i]
        d = distance(points[i-1], p_i)
        if (D + d) >= I:
            qx = points[i-1][0] + ((I-D) / d) * (p_i[0] - points[i-1][0])
            qy = points[i-1][1] + ((I-D) / d) * (p_i[1] - points[i-1][1])
            newPoints.append([qx,qy])
            points.insert(i, [qx,qy])
            D = 0
        else: D = D + d
        i+=1
    return newPoints
#}}}

def pathlength(points):#{{{
    d = 0
    for i,p_i in enumerate(points[:len(points)-1]):
        d += distance(p_i, points[i+1])
    return d
#}}}

def distance(p1, p2): return float(sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2))

def centroid(points): return float(average([float(i[0]) for i in points])), float(average([float(i[1]) for i in points]))

def rotate_to_zero(points):#{{{
    cx, cy = centroid(points)
    theta = atan2(cy - points[0][1], cx - points[0][0])
    newPoints = rotate_by(points, -theta)
    return newPoints
#}}}

def rotate_by(points, theta):#{{{
    cx, cy = centroid(points)
    newpoints = []
    cos_p, sin_p = cos(theta), sin(theta)
    for p in points:
        qx = (p[0] - cx) * cos_p - (p[1] - cy) * sin_p + cx
        qy = (p[0] - cx) * sin_p + (p[1] - cy) * cos_p + cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

def bounding_box(points): #{{{
    minx, maxx = min((p[0] for p in points)), max((p[0] for p in points))
    miny, maxy = min((p[1] for p in points)), max((p[1] for p in points))
    return minx, miny, maxx-minx, maxy - miny 
#}}}

def scale_to_square(points, Size):#{{{
    min_x, min_y, w, h = bounding_box(points)
    newPoints = []
    for p in points:
        qx = p[0] * (float(Size) / w )
        qy = p[1] * (float(Size) / h )
        newPoints.append([qx,qy])
    return newPoints
#}}}

def translate_to_origin(points):#{{{
    cx, cy = centroid(points)
    newpoints = []
    for p in points:
        qx, qy = p[0] - cx , p[1] - cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

psi = .5 * (-1 + sqrt(5.))
def distance_at_best_angle(points, T, ta, tb, td):#{{{
    x1 = psi * ta + (1-psi)*tb
    f1 = distance_at_angle(points, T, x1)
    x2 = (1-psi)*ta + psi*tb
    f2 = distance_at_angle(points, T, x2)
    while abs(tb - ta) > td:
        if f1 < f2:
            tb,x2,f2 = x2, x1, f1
            x1 = psi*ta + (1-psi)*tb
            f1 = distance_at_angle(points, T, x1)
        else:
            ta,x1,f1 = x1, x2, f2
            x2 = (1 - psi)*ta + psi*tb
            f2 = distance_at_angle(points, T, x2)
    return min(f1, f2)
#}}}

def distance_at_angle(points, T, theta):#{{{
    newpoints = rotate_by(points, theta)
    d = pathdistance(newpoints, T)
    return d
#}}}

def pathdistance(a,b):#{{{
    d = 0
    for ai, bi in zip(a,b):
        d += distance(ai, bi)
    return d / len(a)
#}}}

def classify(candidate_points,templates, screen, debug=False):#{{{
    np = resample(candidate_points,64)
    np = rotate_to_zero(np)
    if debug: plot(np, (255,0,0), screen)
    np = scale_to_square(np, Size)
    if debug: plot(np, (255,0,255), screen)
    np = translate_to_origin(np)
    if debug: plot(np, (0,255,255), screen)
    template, score = recognize(np, templates)
    return template
#}}}
    
def plot(points, color, screen, Size=1):#{{{
    for p in points:
        pygame.draw.circle(screen,color, p, 1);
        pygame.display.flip()
#}}}

def load_template(xmlfile):#{{{   Returns a set of points loaded from xml file
    points = []
    p_xml = minidom.parse(open(xmlfile,'r'))
    for tag in p_xml.getElementsByTagName("Point"):
        points.append([int(tag.attributes['X'].value), int(tag.attributes['X'].value)])
    return points
#}}}

def sort_colonne_croissant(x, y):#{{{
    for i in range(len(x)-1, -1, -1):
        if x[i] < y[i]:
            return -1
        elif x[i] > y[i]:
            return 1
    return 0

### Fonction de reconnaisance

In [6]:
def recognize(points, templates):#{{{
    b = float("inf")
    #theta = pi / 4 #45 degrees
    theta = 45.
    #t_d = 2 * (pi / 180)
    t_d = 2.
    Tp = None
    num = 0
    dataOutput = []
    j = 0
    index = []
    for i,T in enumerate(templates):
        #Contien la ligne courante
        listDim1 = []
        #ajoute de le Nom
        listDim1.append(Nom)
        #ajoute de le Prénom
        listDim1.append(Prenom)
        j=j+1
        #ajoute de un index personalisé : ici le numéro de la signature
        index.append('Signature numéro '+ `j`)
        Tpoints = T.points
        #ajoute de la signature du dataset
        listDim1.append(Tpoints)
        #ajoute du nuage de point de la signature à vérifier
        listDim1.append(points)
        d = distance_at_best_angle(points, Tpoints, -theta, theta, t_d)
        #ajoute du nom du template (pour le débug)
        #listDim1.append(T.name)
        #ajoute du score
        listDim1.append(d)
        #ajoute de le précision
        listDim1.append(Precision)
        #met la colonne MOYENNE à vide car elle sert pour le total
        listDim1.append('˅')
        #met la colonne VALIDATION à vide car elle sert pour le total
        listDim1.append('˅')
        #contiendra toutes les lignes du tableau de sortie
        dataOutput.append(listDim1)
        if d < b:
            b = d
            Tp = T
    #création du dataFrame de sortie
    columns = ['Nom', 'Prénom', 'Signature du dataset', 'Signature à tester', 'Score en %', 'Précision', 'MOYENNE', 'VALIDATION']
    moyenne = pd.DataFrame(dataOutput, index=index, columns=columns)["Score en %"].mean()
    if moyenne > Precision :
        print ">>> Cette signature n'est pas celle de " + Nom + " " + Prenom + " (pour une precision de " + `Precision` + ")"
        dataOutput.append(['>', '>', '>', '>', '>', '>', moyenne, 'Non validé'])
    else :
        print ">>> Cette signature est bien celle de " + Nom + " " + Prenom + "(pour une precision de " + `Precision` + ")"
        dataOutput.append(['>', '>', '>', '>', '>', '>', moyenne, 'Validé'])
    index.append('RESULTAT')
    df = pd.DataFrame(dataOutput, index=index, columns=columns)
    print "\n"
    print ">>> La moyenne de chaque véracité du dataset est de : ", moyenne
    print "\n"
    #écriture dans le CSV
    df.to_csv('bddSignature.csv', sep=';', encoding='utf-8')
    result = pd.read_csv('bddSignature.csv', sep=';')
    print "/////////////////////////////////////////////////////////////////////////////////// \n Tableau des résultats de tous les lancements de programme (bdd en quelque sorte) :"
    print result
    return Tp, 1-(b/(0.5 * sqrt(Size*Size * 2)))
#}}}

### Programme principal

In [7]:
def main():
    for t in templates: t.prepare()
    pygame.init();
    screen = pygame.display.set_mode((500,500));
    pygame.display.flip();
    screen.fill((0,0,0))
    running = True
    recording = False
    points = []
    while running:
        events = pygame.event.get();
        for e in events:
            if e.type==MOUSEBUTTONDOWN:
                screen.fill((0,0,0))
                points = []
                recording = True
            if e.type==MOUSEBUTTONUP:
                recording = False
                if len(points) > 10:
                    template_match = classify(points, templates,screen)
                    if template_match : print ">>> La signature du dataset qui s'en approche le plus est la", template_match.name
            if e.type==MOUSEMOTION:
                if recording:
                    mousepos= pygame.mouse.get_pos()
                    points.append(list(mousepos))
                    pygame.draw.circle(screen,(255,255,255), mousepos, 3);
            if e.type==QUIT:
                running = 0
        pygame.time.wait(2);
        pygame.display.flip();
if __name__=="__main__": main()

>>> Cette signature est bien celle de test William(pour une precision de 70)


>>> La moyenne de chaque véracité du dataset est de :  41.8697063566


/////////////////////////////////////////////////////////////////////////////////// 
 Tableau des résultats de tous les lancements de programme (bdd en quelque sorte) :
           Unnamed: 0   Nom   Prénom  \
0  Signature numéro 1  test  William   
1  Signature numéro 2  test  William   
2  Signature numéro 3  test  William   
3  Signature numéro 4  test  William   
4  Signature numéro 5  test  William   
5  Signature numéro 6  test  William   
6            RESULTAT     >        >   

                                Signature du dataset  \
0  [[-161.009366362, 5.68434188608e-14], [-153.98...   
1  [[-146.862898386, 0.0], [-133.838622274, 7.680...   
2  [[-172.838343751, -5.68434188608e-14], [-160.8...   
3  [[-155.857929514, -1.13686837722e-13], [-144.1...   
4  [[-164.899261043, 0.0], [-152.675922621, 5.425...   
5  [[-175.423193101, 3.4

_William GALBY, dans le cadre du cours de Machine Leaning de Jeff Abrahamson - Basé sur l'algo "$1" de Jacob O. Wobbrock, Andrew D. Wilson, Yang Li_